In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim
import pickle
from pyLDAvis.gensim import prepare
from pathlib import Path
from gensim.models import CoherenceModel
import numpy as np

path_base = "/mnt/nfs/scratch1/hshukla/sentence_results/"
model_base = "sen_lda_{}_{}.model"
data_base = "df_sen_{}_{}_tmp.pkl"
risk_label = "item1a_risk"
mda_label = "item7_mda"
start_year = 2012
end_year = 2015
warnings.filterwarnings("ignore", category=DeprecationWarning)

""" Risk and MDA for window=1,5,7 for a total of 6 viz """

' Risk and MDA for window=1,5,7 for a total of 6 viz '

In [2]:
def get_viz(lda_path, data, label, is_filter_extreme, ws):
    lda_model = LdaModel.load(lda_path)
    
    # Load documents
    docs_path = Path(path_base + "{}_{}_{}_{}_{}.pkl".format("documents",start_year,end_year,label,ws))    
    if docs_path.exists():
        display("Loading Documents")
        with open(docs_path, "rb") as file:
            documents = pickle.load(file)        
    else:
        display("Parsing Documents")
        data_slice = data[(data.year_x >= start_year) & (data.year_x <= end_year)]
        documents = [sentence_grp for doc in data_slice[label].to_list() for sentence_grp in doc]
        with open(docs_path, "wb") as file:
            pickle.dump(documents, file)
    
    # Load dictionary
    dict_path = Path(path_base + "{}_{}_{}_{}_{}.pkl".format("dictionary",start_year,end_year,label,ws))
    if dict_path.exists():
        display("Loading Dict")
        with open(dict_path, "rb") as file:
            dictionary = pickle.load(file)
    else:
        display("Parsing Dict")
        dictionary = Dictionary(documents)
        if is_filter_extreme:
            dictionary.filter_extremes(no_below=10)
        with open(dict_path, "wb") as file:
            pickle.dump(dictionary, file)
    
    
    # Load corpus
    corpus_path = Path(path_base + "{}_{}_{}_{}_{}.pkl".format("corpus",start_year,end_year,label,ws))
    if corpus_path.exists():
        display("Loading Corpus")
        with open(corpus_path, "rb") as file:
            corpus = pickle.load(file)
    else:
        display("Parsing Corpus")
        corpus = [dictionary.doc2bow(doc) for doc in documents]
        _temp = dictionary[0] # Initialize id2token mappings
        id2word = dictionary.id2token
        with open(corpus_path, "wb") as file:
            pickle.dump(corpus, file)
            
    viz_model = prepare(lda_model, corpus, dictionary=dictionary)
    pyLDAvis.save_html(viz_model, "{}_{}_{}_{}_{}.html".format("ldaviz",start_year,end_year,label,ws))
    return viz_model
    
    
    
def get_coh(lda_path, data, label, is_filter_extreme, ws):
    lda_model = LdaModel.load(lda_path)
    
    # Load documents
    docs_path = Path(path_base + "{}_{}_{}_{}_{}.pkl".format("documents",start_year,end_year,label,ws))    
    if docs_path.exists():
        with open(docs_path, "rb") as file:
            documents = pickle.load(file)        
    else:
        data_slice = data[(data.year_x >= start_year) & (data.year_x <= end_year)]
        documents = [sentence_grp for doc in data_slice[label].to_list() for sentence_grp in doc]
        with open(docs_path, "wb") as file:
            pickle.dump(documents, file)
    
    # Load dictionary
    dict_path = Path(path_base + "{}_{}_{}_{}_{}.pkl".format("dictionary",start_year,end_year,label,ws))
    if dict_path.exists():
        with open(dict_path, "rb") as file:
            dictionary = pickle.load(file)
    else:
        dictionary = Dictionary(documents)
        if is_filter_extreme:
            dictionary.filter_extremes(no_below=10)
        with open(dict_path, "wb") as file:
            pickle.dump(dictionary, file)
    
    
    # Load corpus
    corpus_path = Path(path_base + "{}_{}_{}_{}_{}.pkl".format("corpus",start_year,end_year,label,ws))
    if corpus_path.exists():
        with open(corpus_path, "rb") as file:
            corpus = pickle.load(file)
    else:
        corpus = [dictionary.doc2bow(doc) for doc in documents]
        _temp = dictionary[0] # Initialize id2token mappings
        id2word = dictionary.id2token
        with open(corpus_path, "wb") as file:
            pickle.dump(corpus, file)
    
    cm_mass = CoherenceModel(model=lda_model, corpus=corpus, dictionary=dictionary, coherence='u_mass')
    cm_cv = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_v')
    cm_cuci = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_uci')
    cm_npmi = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_npmi')
    
    display("Umass: {:+.4f}".format(cm_mass.get_coherence()))
    display("CV: {:+.4f}".format(cm_cv.get_coherence()))
    display("CUCI: {:+.4f}".format(cm_cuci.get_coherence()))
    display("NPMI: {:+.4f}".format(cm_npmi.get_coherence()))

# WindowSize = 1, WindowOverlap = 1

In [3]:
one_data = pd.read_pickle(path_base + data_base.format(1,1))

#### Risk

In [4]:
one_model_base = "window1_rerunsen_lda_{}_{}.model"
one_risk_lda_path = path_base + one_model_base.format(risk_label,1)

In [5]:
viz = get_viz(one_risk_lda_path, one_data, risk_label, True, 1)

'Loading Documents'

'Loading Dict'

'Loading Corpus'

In [ ]:
get_coh(one_risk_lda_path, one_data, risk_label, True, 1)

#### MD

In [5]:
one_mda_lda_path = path_base + one_model_base.format(mda_label,1)

In [6]:
viz = get_viz(one_mda_lda_path, one_data, mda_label, True, 1)

'Loading Documents'

'Loading Dict'

'Loading Corpus'

In [ ]:
get_coh(one_mda_lda_path, one_data, mda_label, True, 1)

# WindowSize = 5, WindowOverlap = 2

In [ ]:
five_data = pd.read_pickle(path_base + data_base.format(5,2))

#### Risk

In [ ]:
five_risk_lda_path = path_base + model_base.format(risk_label,5)

In [ ]:
viz = get_viz(five_risk_lda_path, five_data, risk_label, True, 5)

In [ ]:
get_coh(five_risk_lda_path, five_data, risk_label, True, 5)

#### MD&A

In [ ]:
five_mda_lda_path = path_base + model_base.format(mda_label,5)

In [ ]:
viz = get_viz(five_mda_lda_path, five_data, mda_label, True, 5)

In [ ]:
get_coh(five_mda_lda_path, five_data, mda_label, True, 5)

## WindowSize = 7, WindowOverlap = 3

In [ ]:
del five_data
seven_data = pd.read_pickle(path_base + data_base.format(7,3))

#### Risk

In [ ]:
seven_risk_lda_path = path_base + model_base.format(risk_label,7)

In [ ]:
viz = get_viz(seven_risk_lda_path, seven_data, risk_label, True, 7)

In [ ]:
get_coh(seven_risk_lda_path, seven_data, risk_label, True, 7)

#### MD&A

In [ ]:
seven_mda_lda_path = path_base + model_base.format(mda_label,7)

In [ ]:
viz = get_viz(seven_mda_lda_path, seven_data, mda_label, True, 7)

In [ ]:
get_coh(seven_mda_lda_path, seven_data, mda_label, True, 7)

In [ ]:
item1a = [-4.0796, -2.4353, -2.0128]
item7 = [-5.2741, 0.5924, -2.4623]

x = []
labels = ["1", "5", "7"]
x = np.arange(len(labels))
title="Sentence Coherence - Umass"
width = 0.35  # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(x - width / 2, item1a, width, label='Item1a')
rects2 = ax.bar(x + width / 2, item7, width, label='Item7')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Coherence Score')
ax.set_xlabel('Window Size')
ax.set_title(title)
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
# fig.tight_layout()
plt.show()

In [ ]:
item1a = [0.5407, 0.5097, 0.5316]
item7 = [0.4854, 0.5924, 0.5446]

x = []
labels = ["1", "5", "7"]
x = np.arange(len(labels))
title="Sentence Coherence - C_v"
width = 0.35  # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(x - width / 2, item1a, width, label='Item1a')
rects2 = ax.bar(x + width / 2, item7, width, label='Item7')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Coherence Score')
ax.set_xlabel('Window Size')
ax.set_title(title)
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
# fig.tight_layout()
plt.show()